# Run the pipeline classification by file OR with cells below

## By file

In [ ]:

import sys
sys.path.insert(1, 'C:\\Users\\jamilah\\Documents\\Github_analysis_PROJECTS\\Motor_classification\\')

from e_classification.main_script import *

# ------------------------------
# Run all classification combinations by a single file
# ------------------------------
main_script()

## By cells (for troubleshooting and it runs 4 to 6 times faster than the textfile with interpretor)

In [1]:
import pickle
import numpy as np
import pandas as pd
import time

from scipy import signal
from collections import Counter

from sklearn.model_selection import train_test_split

import sys
sys.path.insert(1, 'C:\\Users\\jamilah\\Documents\\Github_analysis_PROJECTS\\Motor_classification')
from c_calculate_metrics.put_timeseries_trialdata_into_pandas import *

# Personal python functions
from subfunctions.make_a_properlist import *
from subfunctions.numderiv import *
from subfunctions.freqresp_functions import get_freqresp_mag_phase, select_fc, select_filter
from subfunctions.freq_from_sig_timecounting import *
from subfunctions.freq_from_sig_freqresp import *
from subfunctions.my_dropna_python import *
from subfunctions.scale_feature_data import *
from subfunctions.normal_distribution_feature_data import *
from subfunctions.findall import *
from subfunctions.explode_without_colnames2 import *
from subfunctions.scikit_functions_binaryclass import *
from subfunctions.scikit_functions import *

df_timeseries_exp = put_timeseries_trialdata_into_pandas()

In [ ]:
print('shape of df_timeseries_exp[rot] : ', df_timeseries_exp['rot'].shape)
df_timeseries_exp['rot'].tail()

In [ ]:
print('shape of df_timeseries_exp[trans] : ', df_timeseries_exp['trans'].shape)
df_timeseries_exp['trans'].tail()

In [2]:
def get_df(ax_val, ss_val):
    
    if ax_val == 'all' and ss_val == 'all':
        # All the data
        df_timeseries_exp[exp].head()
        df = df_timeseries_exp[exp]
    elif ax_val != 'all' and ss_val == 'all':
        # Prediction for each axis
        if ax_val == 'ax0':
            ax_val_n = 0
        elif ax_val == 'ax1':
            ax_val_n = 1
        elif ax_val == 'ax2':
            ax_val_n = 2
        df = df_timeseries_exp[exp][(df_timeseries_exp[exp].ax == ax_val_n)]
    elif ax_val == 'all' and ss_val != 'all':
        # Prediction per sup/sub
        if ss_val == 'sup':  # sup
            df = df_timeseries_exp[exp][(df_timeseries_exp[exp].ss > 0)]
        elif ss_val == 'sub':  # sub
            df = df_timeseries_exp[exp][(df_timeseries_exp[exp].ss < 0)]
    elif ax_val != 'all' and ss_val != 'all':
        # Prediction per axis and sup/sub
        if ax_val == 'ax0':
            ax_val_n = 0
        elif ax_val == 'ax1':
            ax_val_n = 1
        elif ax_val == 'ax2':
            ax_val_n = 2
        
        if ss_val == 'sup':  # sup
            df = df_timeseries_exp[exp][(df_timeseries_exp[exp].ss > 0) & (df_timeseries_exp[exp].ax == ax_val_n)]
        elif ss_val == 'sub':  # sub
            df = df_timeseries_exp[exp][(df_timeseries_exp[exp].ss < 0) & (df_timeseries_exp[exp].ax == ax_val_n)]

    print('Confirmation : exp=', exp, ', ax_val=', ax_val, ', ss_val=', ss_val)
    
    # Reset the index
    df.reset_index(drop=True, inplace=True)
    
    return df

In [3]:
# Get the joystick stimulus axis data and put it in a pandas column
def indexit(row):
    joy_mat = [row.JOY_ax0, row.JOY_ax1, row.JOY_ax2]
    return joy_mat[int(row.ax)]

In [4]:
def create_labels_and_initial_feature(df):
    # IC = 1
    # EC = 2, 4, 5
    # NC = 3, 6, 7
    # NR = 9
    # (IC) - sham (do not use) = 8
    # (NC) - sham (do not use) = 10

    # Just to confirm, what are the unique values of res_type
    df.res_type.value_counts(ascending=True)

    # Construction of SD_label : How do we define disorientation?

    # Way 0 : lenient
    # 0 = If participates got the result CORRECT for the trial, they were NOT disoriented. (IC, EC)
    # 1 = If participates got the result WRONG or did not respond, they were disoriented. (NC, NR)

    idx_NDS = df.index[(df.res_type == 1) | (df.res_type == 2) | (df.res_type == 4) | (df.res_type == 5)].to_list()
    idx_DS = df.index[(df.res_type == 3) | (df.res_type == 6) | (df.res_type == 7) | (df.res_type == 9)].to_list()
    df.lenient = ''  # define a new column , rows 8 and 10 will be NaN, need to do dropna for rows
    df.loc[idx_NDS, 'lenient'] = 0
    df.loc[idx_DS, 'lenient'] = 1

    # -------------------------------------

    # Way 1 : strict simple
    # 0 = If participants got the result initially CORRECT, they were NOT disoriented. (IC)
    # 1 = If participants were WRONG at any point, they were disoriented. (EC, NC, NR)

    idx_NDS = df.index[(df.res_type == 1)].to_list()
    idx_DS = df.index[(df.res_type == 2) | (df.res_type == 4) | (df.res_type == 5) | (df.res_type == 3) | (df.res_type == 6) | (df.res_type == 7) | (df.res_type == 9)].to_list()
    df.strict = ''  # define a new column
    df.loc[idx_NDS, 'strict'] = 0
    df.loc[idx_DS, 'strict'] = 1

    # Way 2 : st_complex
    # 0 = If participants got the result initially CORRECT, they were NOT disoriented. (IC)
    # 1 = If participants got the result eventually CORRECT, they were MILDLY disoriented. (EC)
    # 2 = If participants were WRONG for the trial, they were disoriented. (NC, NR)

    idx_NDS = df.index[(df.res_type == 1)].to_list()
    idx_MDS = df.index[(df.res_type == 2) | (df.res_type == 4) | (df.res_type == 5)].to_list()
    idx_DS = df.index[(df.res_type == 3) | (df.res_type == 6) | (df.res_type == 7) | (df.res_type == 9)].to_list()
    df.st_complex = ''  # define a new column
    df.loc[idx_NDS, 'st_complex'] = 0
    df.loc[idx_MDS, 'st_complex'] = 1
    df.loc[idx_DS, 'st_complex'] = 2

    # -------------------------------------

    # Create features :  (1) position
    df['joy_stim'] = df.apply(indexit, axis='columns')  # fill in joy_stim
    
    # -------------------------------------

    # Make DataFrame for trial start-stop index
    # Cut the data up per trial across subjects
    tr_vec = df.tr.to_numpy()

    st = [0]
    ender = []
    for i in range(len(tr_vec)-1):
        if tr_vec[i] != tr_vec[i+1]:
            st = st + [i+1]
            ender = ender + [i]
    ender = ender + [len(tr_vec)-1]

    # See start-stop index clearly
    e0 = np.reshape(st, (len(st),1))
    e1 = np.reshape(ender, (len(st),1))
    data = np.ravel(e0), np.ravel(e1)
    data = np.transpose(data)
    columns = ['stind', 'endind']
    temp = pd.DataFrame(data=data, columns=columns)

    # -------------------------------------

    # Find the longest trial signal in df_rot['joy_stim']
    temp['diff'] = temp.endind - temp.stind
    temp['timediff'] = [df.time.iloc[temp.endind[i]] - df.time.iloc[temp.stind[i]] for i in range(len(temp.endind))]
    outmin = temp['diff'].min()
    outmax = temp['diff'].max()

    tomin = temp['timediff'][(temp['diff'] == outmin)]
    tomax = temp['timediff'][(temp['diff'] == outmax)]
    # print('outmin : ', outmin, 't :', tomin)
    # print('outmax : ', outmax, 't :', tomax)

    # -------------------------------------

    # Interpolate : make each trial the same number of data points
    from scipy.interpolate import interp1d
    feat0 = []
    t_feat0 = []
    y1_feat0 = []
    y2_feat0 = []
    y3_feat0 = []
    for i in range(len(temp.stind)):
        
        # X
        sSIG = df['joy_stim'][temp.stind.iloc[i]:temp.endind.iloc[i]].to_numpy()
        t_sSIG = df['time'][temp.stind.iloc[i]:temp.endind.iloc[i]].to_numpy()

        # labels
        y1 = df['lenient'][temp.stind.iloc[i]:temp.endind.iloc[i]].to_numpy()
        y2 = df['strict'][temp.stind.iloc[i]:temp.endind.iloc[i]].to_numpy()
        y3 = df['st_complex'][temp.stind.iloc[i]:temp.endind.iloc[i]].to_numpy()
        
        # Check if trial data is less than the maximum length
        if len(df['joy_stim'][temp.stind.iloc[i]:temp.endind.iloc[i]]) < outmax:
            
            # The trial length is different so interpolate the time-series to make them the same length signal 
            x = np.linspace(sSIG[0], len(sSIG), num=len(sSIG), endpoint=True)
            xnew = np.linspace(sSIG[0], len(sSIG), num=outmax, endpoint=True)

            # joystick on stim
            f = interp1d(x, sSIG)
            sSIGl = f(xnew)

            # time
            f = interp1d(x, t_sSIG)
            t_sSIGl = f(xnew)

            # y1
            f = interp1d(x, y1)
            y1_sSIGl = f(xnew)

            # y2
            f = interp1d(x, y2)
            y2_sSIGl = f(xnew)

            # y3
            f = interp1d(x, y3)
            y3_sSIGl = f(xnew)

            # python : you can not create a matrix in real-time in pandas
            # you only assign the full matrix at the end
            # (0) position
            feat0 = feat0 + [sSIGl]
            t_feat0 = t_feat0 + [t_sSIGl]
            y1_feat0 = y1_feat0 + [np.ravel(y1_sSIGl)]
            y2_feat0 = y2_feat0 + [np.ravel(y2_sSIGl)]
            y3_feat0 = y3_feat0 + [np.ravel(y3_sSIGl)]
            
            del x, f, sSIGl, t_sSIGl, y1_sSIGl, y2_sSIGl, y3_sSIGl
        else:
            feat0 = feat0 + [sSIG]
            t_feat0 = t_feat0 + [t_sSIG]
            y1_feat0 = y1_feat0 + [np.ravel(y1)]
            y2_feat0 = y2_feat0 + [np.ravel(y2)]
            y3_feat0 = y3_feat0 + [np.ravel(y3)]

    # Clean up
    del df
    # -------------------------------------
    
    return feat0, t_feat0, y1_feat0, y2_feat0, y3_feat0

In [5]:
def feature_creation_preprocessing(feat0, t_feat0):

    start = time.time()
    
    # ----------------
    # Make your features
    # ----------------
    norm_feat0 = []
    norm_feat1 = []
    norm_feat2 = []
    norm_feat3 = []
    feat4 = []
    feat4_1 = []
    
    n = 4   # filter order
    fs = 250 # data sampling frequency (Hz)
    fcc = 10  # Cut-off frequency of the filter
    w = fcc / (fs / 2) # Normalize the frequency
    b, a = signal.butter(n, w, 'low')  # 3rd order

    # Need to find when one trial starts and end - take derivative from start-stop periods
    for i in range(len(feat0)):

        if i == 0:
            plotORnot = 0 # 1
        else:
            plotORnot = 0

        # (0) position
        scaled_data0 = scale_feature_data(feat0[i], plotORnot)
        # print('shape of scaled_data0 : ', np.array(scaled_data0).shape)
        temp0 = normal_distribution_feature_data(scaled_data0, plotORnot)
        norm_feat0 = norm_feat0 + [temp0]

        # (1) velocity
        vel = numderiv(feat0[i], t_feat0[i])
        scaled_data1 = scale_feature_data(vel, plotORnot)
        temp1 = normal_distribution_feature_data(scaled_data1, plotORnot)
        norm_feat1 = norm_feat1 + [temp1]

        # (2) acceleration
        # the signal is noisy, there is a numerical defect : 1) tried changing the time to see if it would smooth out (does not work), 2) filter

        # shift time to the right by one
        # t_shift = []
        # for b in range(len(t_feat0[i])-1):
            # t_shift.append(t_feat0[i][b+1])
        # t_shift.append(t_feat0[i][-1])
        # acc = numderiv(vel, t_shift)
        acc = numderiv(vel, t_feat0[i])
        filtacc = signal.filtfilt(b, a, acc)

        scaled_data2 = scale_feature_data(filtacc, plotORnot)
        temp2 = normal_distribution_feature_data(scaled_data2, plotORnot)
        norm_feat2 = norm_feat2 + [temp2]

        # (3) frequency response of joystick via FFT
        Xn_mag_in, Xn_phase_in, omeg_in, Xn_mag_half_db_in, Xn_phase_half_in, omg_half_in = get_freqresp_mag_phase(feat0[i], t_feat0[i], 0.1)
        scaled_data3 = scale_feature_data(Xn_mag_in, plotORnot)
        temp3 = normal_distribution_feature_data(scaled_data3, plotORnot)
        norm_feat3 = norm_feat3 + [temp3]

        # (4) frequency of joystick : frequency response db cutoff
        dp_jump = 0.5  # this is half the height of the scaled signal scaled_data0
        fc = freq_from_sig_timecounting(scaled_data0, t_feat0[i], 0.1, dp_jump, plotORnot=0)
        fc1 = freq_from_sig_freqresp(feat0[i], t_feat0[i], ts=0.1, plotORnot=0)
        temp4 = fc*np.ones((len(feat0[i])))
        feat4 = feat4 + [temp4]

        temp4_1 = fc1*np.ones((len(feat0[i])))
        feat4_1 = feat4_1 + [temp4_1]

        # print('fc = ', fc, ', fc1 = ', fc1)

        # (5) integral error with respect to theoretical stimuli (this is really part of the control response)
        # diff = feat0[i] - theoretical_stim[i]
        # feat5 = feat5 + [diff]

    # Clean up
    del n, fs, fcc, w, b, a, plotORnot
    del feat0, t_feat0, scaled_data0, vel, scaled_data1, scaled_data2, filtacc
    del Xn_mag_in, Xn_phase_in, omeg_in, Xn_mag_half_db_in, Xn_phase_half_in, omg_half_in, scaled_data3
    del fc, fc1
    # ----------------

    end = time.time()
    print('Elasped time for feature processing : ', end - start)

    return norm_feat0, norm_feat1, norm_feat2, norm_feat3, feat4, feat4_1

In [18]:
def pad_data_2makeclasses_equivalent(df_test):

    # ----------------

    # Remove nan value per row
    # df_test.dropna(axis=0)
    # OR
    df_test_noNan = my_dropna_python(df_test)

    #del df_test
    
    # ----------------
    
    # DataFrame is organized : [X, y]  where each of X columns are nested arrays

    num_of_feat = df_test_noNan.shape[1] - 1

    # ----------------

    # Add on a scalar column of the y label, for searching
    val = []
    for i in range(len(df_test_noNan.iloc[:,-1])):
        val.append(df_test_noNan.iloc[i,-1][0])
    # print('length of val : ', len(val))
    valS = pd.Series(val)

    df_test_noNan = pd.concat([df_test_noNan, valS], axis=1)
    df_test_noNan = df_test_noNan.rename({0: 'y_scalar'}, axis=1)

    # ----------------

    # Count the unique values in the label
    df_vc = pd.DataFrame(df_test_noNan.y_scalar.value_counts())
    df_vc.head()
    counted_value = df_vc.iloc[:,0].to_numpy()
    count_index = df_vc.iloc[:,0].index.to_numpy()
    # print('Before sorting counted_value : ', counted_value)
    # print('Before sorting count_index : ', count_index)
    
    # Sort counted_value by count_index; in ascending order
    sind = np.argsort(count_index)
    count_index = count_index[sind]
    counted_value = counted_value[sind]
    print('After sorting counted_value : ', counted_value)
    print('After sorting count_index : ', count_index)
    
    out = {}
    for i, val in enumerate(counted_value):
        out[count_index[i]] = val
    print('Original class count : ', out)

    # ----------------

    # Determine how much to pad each class label
    needed_samps_class = np.max(counted_value) - counted_value
    print('needed_samps_class : ', needed_samps_class)

    # ----------------

    # Confirm that there are no nan values
    out = df_test_noNan.isnull().values.any()
    print('Are there nan valeus in the data : ', out)

    # ----------------
    
    print('shape of dataframe before padding : ', df_test_noNan.shape)

    # ----------------
    # Pad the DataFrame
    n_classes = len(count_index)
    n_samples = len(df_test_noNan)

    df_2add_on = pd.DataFrame()

    for i in range(n_classes):
        #print('i : ', i)
        # Pad short length classes
        for j in range(needed_samps_class[i]):
            #print('j : ', j) 
            flag = 0
            while flag == 0:
                permvec = np.random.permutation(n_samples)
                index = permvec[0]  #random choosen index

                # look for each class
                if i == int(df_test_noNan.y_scalar.iloc[index]):
                    #print('Class match was found : i = ', i, ', data index = ', int(df_test_noNan.y_scalar.iloc[index]), ', index = ', index)
                    
                    # Append the data with padded data entry
                    data = []
                    for nf in range(num_of_feat+2):
                        if nf == num_of_feat+1:
                            # The last column is a scalar - can not use list
                            data.append([df_test_noNan.iloc[index, nf]])
                        else:
                            data.append([list(df_test_noNan.iloc[index, nf])])

                    df_row = pd.DataFrame(data=data)
                    df_row = df_row.T # or df1.transpose()
                    df_2add_on = pd.concat([df_2add_on, df_row], axis=0)

                    flag = 1 # to brake while

    # ----------------

    df_2add_on = df_2add_on.reset_index(drop=True)  # reset index : delete the old index column

    dfl = {}
    for nf in range(num_of_feat+2):
        if nf == num_of_feat+1:
            dfl[nf] = 'y_scalar'
        elif nf == num_of_feat:
            dfl[nf] = 'y'
        else:
            dfl[nf] = '%d' % (nf)

    df_2add_on = df_2add_on.rename(dfl, axis=1)
    
    print('shape of dataframe to add to original dataframe: ', df_2add_on.shape)

    # ----------------

    # want to arrange the dataframe with respect to rows (stack on top of the other): so axis=0 
    # OR think of it as the rows of the df change so you put axis=0 for rows
    df_test2 = pd.concat([df_test_noNan, df_2add_on], axis=0)
    df_test2 = df_test2.reset_index(drop=True)  # reset index : delete the old index column

    print('shape of padded dataframe (original + toadd) : ', df_test2.shape)
    
    del df_test_noNan, df_2add_on

    # ----------------

    # Final check of class balance
    df_vc = pd.DataFrame(df_test2.y_scalar.value_counts())
    df_vc.head()
    counted_value = df_vc.iloc[:,0].to_numpy()
    count_index = df_vc.iloc[:,0].index.to_numpy()
    # print('Before sorting counted_value : ', counted_value)
    # print('Before sorting count_index : ', count_index)
    
    # Sort counted_value by count_index; in ascending order
    sind = np.argsort(count_index)
    count_index = count_index[sind]
    counted_value = counted_value[sind]
    print('After sorting counted_value : ', counted_value)
    print('After sorting count_index : ', count_index)
    
    out = {}
    for i, val in enumerate(counted_value):
        out[count_index[i]] = val
    print('New class count : ', out)

    # Determine how much to pad each class label
    needed_samps_class = np.max(counted_value) - counted_value
    print('needed_samps_class : ', needed_samps_class)

    # ----------------
    
    # After the final check, drop the y_scalar column
    df_test2 = df_test2.drop('y_scalar', 1)  # 1 is the axis number (0 for rows and 1 for columns)
    
    # ----------------

    return df_test2

In [19]:
# Plotting columns from featlab: pandas
plotORnot = 0
if plotORnot == 1:
    # Plot all the data (data is too large : can not do it)
    pd.options.plotting.backend = "plotly"
    fig = featlab.loc[:,8].plot()
    fig.show()

In [20]:
def preprocess_df_2_XYtraintest(featlab_new, fea, ynum, ovtot):
    # ----------------
    # Check to see if each y label has a balanced number of class samples
    # ----------------
    num_of_feat = featlab_new.shape[1] - 1 #6
    test_size = 0.25 # default

    # ----------------
    # Order of which features to use in a model : 1) All features, 
    # 2) first 3 from permutation_importance, 
    # 3) first 2 from permutation_importance, 
    # 4) first feature from permutation_importance
    if fea == 0:
        # 1) All features
        X_part = list(range(num_of_feat))
        y_part = num_of_feat   # Add label y
        desired_col = make_a_properlist([X_part, y_part])
    elif fea == 1:
        # 2) first 3 from permutation_importance
        desired_col = []
        # Add features for X
        for oo in range(3): 
            if oo < len(ovtot):
                if ovtot[oo][0] == 'joy':
                    desired_col.append(0)
                elif ovtot[oo][0] == 'joy1derv':
                    desired_col.append(1)
                elif ovtot[oo][0] == 'joy2derv':
                    desired_col.append(2)
                elif ovtot[oo][0] == 'fres':
                    desired_col.append(3)
                elif ovtot[oo][0] == 'freq_t':
                    desired_col.append(4)
                elif ovtot[oo][0] == 'freq_fres':
                    desired_col.append(5)
            else:
                z = desired_col[-1]
                desired_col.append(z)
        
        desired_col.append(num_of_feat)  # Add label y
        desired_col = np.ravel(desired_col)
    elif fea == 2:
        # 3) first 2 from permutation_importance
        desired_col = []
        for oo in range(2):
            if oo < len(ovtot):
                if ovtot[oo][0] == 'joy':
                    desired_col.append(0)
                elif ovtot[oo][0] == 'joy1derv':
                    desired_col.append(1)
                elif ovtot[oo][0] == 'joy2derv':
                    desired_col.append(2)
                elif ovtot[oo][0] == 'fres':
                    desired_col.append(3)
                elif ovtot[oo][0] == 'freq_t':
                    desired_col.append(4)
                elif ovtot[oo][0] == 'freq_fres':
                    desired_col.append(5)
            else:
                z = desired_col[-1]
                desired_col.append(z)
        
        desired_col.append(num_of_feat)  # Add label y
        desired_col = np.ravel(desired_col)
    elif fea == 3:
        # 4) first feature from permutation_importance
        if ovtot[0][0] == 'joy':
            vvv = 0
        elif ovtot[0][0] == 'joy1derv':
            vvv = 1
        elif ovtot[0][0] == 'joy2derv':
            vvv = 2
        elif ovtot[0][0] == 'fres':
            vvv = 3
        elif ovtot[0][0] == 'freq_t':
            vvv = 4
        elif ovtot[0][0] == 'freq_fres':
            vvv = 5
        y_part = num_of_feat    # Add label y
        desired_col = [vvv, y_part]

    print('desired_col : ', desired_col)  # where first columns are X, and last column is y
    df_y = featlab_new.iloc[:, desired_col]

    # ----------------
    
    print('shape of df_y : ', df_y.shape)
    df_y.head()

    # ----------------

    # Divide DataFrame into X and y
    a = df_y.shape[1]
    print('a : ', a)

    X = df_y.iloc[:,0:a-1]
    X = X.to_numpy()
    print('shape of X : ', X.shape)

    y = df_y.iloc[:,a-1:a].to_numpy()
    y = np.reshape(y, (len(y), 1))
    print('length of y : ', len(y))

    # Check if the y label is correct (only 0's and 1's)
    print('y :', y)

    # ----------------

    # Split the X an y data into test and train
    seed = 0
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state = seed, test_size = test_size)
    print('X_train : ', len(X_train))

    X_train = np.array(X_train)
    print('shape of X_train : ', X_train.shape)
    Y_train = np.array(Y_train)
    print('shape of Y_train : ', Y_train.shape)
    X_test = np.array(X_test)
    print('shape of X_test : ', X_test.shape)
    Y_test = np.array(Y_test)
    print('shape of Y_test : ', Y_test.shape)
    
    # ----------------
    
    # Ensure that Y_train and Y_test are integers
    Y_test = Y_test.astype(int)
    Y_train = Y_train.astype(int)
    
    return X_train, X_test, Y_train, Y_test, desired_col

In [21]:
def classify(featlab_new, ynum):
    
    # Loop over each model to test
    # 0,1,2,3,4,5,6,
    # 7 SVM can not converge
    for wm in [2, 3, 4]:
        res_permod = []
        
        ovtot = []
        
        for fea in range(4):
            print('fea : ', fea)

            # Selects the feature columns from the Dataframe per fea 
            X_train, X_test, Y_train_1D, Y_test_1D, desired_col = preprocess_df_2_XYtraintest(featlab_new, fea, ynum, ovtot)
            print('preprocess_df_2_XYtraintest finished ')
            
            # Determine if classes are binary or multiclass:
            class_len = len(np.unique(Y_train_1D))
            if class_len <= 2:
                cltype = 'binary'
            elif class_len > 2:
                cltype = 'multi'
            
            print('class_len : ', class_len)
            print('cltype : ', cltype)
            
            if wm == 0:
                # Stochastic Gradient Descent
                m_name = 'SGD'
                if cltype == 'binary':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_1D_score, Y_test_1D_score = binary_stochastic_gradient_descent(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_binary_class(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_1D_score)
                    value_pack_test = evaluation_methods_binary_class(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_1D_score)
                elif cltype == 'multi':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_bin_score, Y_test_bin_score = multiclass_stochastic_gradient_descent(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_multi_class_1D(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_bin_score)
                    value_pack_test = evaluation_methods_multi_class_1D(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_bin_score)
            elif wm == 1:
                # Linear Discriminant Analysis (LDA)
                m_name = 'LDA'
                if cltype == 'binary':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_1D_score, Y_test_1D_score = binary_LDA_classifier(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_binary_class(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_1D_score)
                    value_pack_test = evaluation_methods_binary_class(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_1D_score)
                elif cltype == 'multi':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_bin_score, Y_test_bin_score = multiclass_LDA_classifier(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_multi_class_1D(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_bin_score)
                    value_pack_test = evaluation_methods_multi_class_1D(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_bin_score)
            elif wm == 2:
                # RandomForest
                m_name = 'RF'
                if cltype == 'binary':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_1D_score, Y_test_1D_score = binary_RandomForest(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_binary_class(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_1D_score)
                    value_pack_test = evaluation_methods_binary_class(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_1D_score)
                elif cltype == 'multi':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_bin_score, Y_test_bin_score = multiclass_RandomForest_1Dinput(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_multi_class_1D(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_bin_score)
                    value_pack_test = evaluation_methods_multi_class_1D(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_bin_score)
            elif wm == 3:
                # Gradient Boosting Classifier (gradient descent w/ logistic regression cost function)
                m_name = 'GBC'
                if cltype == 'binary':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_1D_score, Y_test_1D_score = binary_GradientBoostingClassifier(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_binary_class(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_1D_score)
                    value_pack_test = evaluation_methods_binary_class(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_1D_score)
                elif cltype == 'multi':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_bin_score, Y_test_bin_score = multiclass_GradientBoostingClassifier(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_multi_class_1D(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_bin_score)
                    value_pack_test = evaluation_methods_multi_class_1D(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_bin_score)
            elif wm == 4:
                # Decision Tree Classifier
                m_name = 'DT'
                if cltype == 'binary':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_1D_score, Y_test_1D_score = binary_Decision_Tree_Classifier(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_binary_class(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_1D_score)
                    value_pack_test = evaluation_methods_binary_class(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_1D_score)
                elif cltype == 'multi':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_bin_score, Y_test_bin_score = multiclass_Decision_Tree_Classifier(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_multi_class_1D(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_bin_score)
                    value_pack_test = evaluation_methods_multi_class_1D(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_bin_score)
            elif wm == 5:
                # Multilayer perceptron (MLP)/neural network (Deep Learning) : logistic regression NN
                m_name = 'MLP'
                if cltype == 'binary':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_1D_score, Y_test_1D_score = binary_multilayer_perceptron(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_binary_class(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_1D_score)
                    value_pack_test = evaluation_methods_binary_class(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_1D_score)
                elif cltype == 'multi':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_bin_score, Y_test_bin_score = multiclass_multilayer_perceptron_1Dinput(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_multi_class_1D(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_bin_score)
                    value_pack_test = evaluation_methods_multi_class_1D(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_bin_score)
            elif wm == 6:
                # Gaussian Naive Bayes
                m_name = 'GNB'
                if cltype == 'binary':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_1D_score, Y_test_1D_score = binary_gaussian_naive_bayes(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_binary_class(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_1D_score)
                    value_pack_test = evaluation_methods_binary_class(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_1D_score)
                elif cltype == 'multi':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_bin_score, Y_test_bin_score = multiclass_gaussian_naive_bayes(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_multi_class_1D(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_bin_score)
                    value_pack_test = evaluation_methods_multi_class_1D(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_bin_score)
            elif wm == 7:
                # Support Vector Machine (NuSVC)
                m_name = 'NuSVC'
                if cltype == 'binary':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_1D_score, Y_test_1D_score = binary_svm_NuSVC(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_binary_class(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_1D_score)
                    value_pack_test = evaluation_methods_binary_class(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_1D_score)
                elif cltype == 'multi':
                    model, Y_train_1D_predict, Y_test_1D_predict, Y_train_bin_pp, Y_test_bin_pp, Y_train_bin_score, Y_test_bin_score = multiclass_svm_NuSVC(X_train, X_test, Y_train_1D, Y_test_1D)
                    value_pack_train = evaluation_methods_multi_class_1D(model, X_train, Y_train_1D, Y_train_1D_predict, Y_train_bin_pp, Y_train_bin_score)
                    value_pack_test = evaluation_methods_multi_class_1D(model, X_test, Y_test_1D, Y_test_1D_predict, Y_test_bin_pp, Y_test_bin_score)
                    
            print('m_name : ', m_name)
            
            # print('value_pack_train : ', value_pack_train)
            print('value_pack_test : ', value_pack_test)
            
            # The cross validation sometimes returned nan, whereas the direct calculation returned a resonable value
            # Thus, taking direct calculation. 
            acc_train = value_pack_train['acc_dircalc']
            prec_train = value_pack_train['prec_dircalc']
            recall_train = value_pack_train['recall_dircalc']
            roc_auc_train = value_pack_train['rocauc_pp_dircalc']

            acc_test = value_pack_test['acc_dircalc']
            prec_test = value_pack_test['prec_dircalc']
            recall_test = value_pack_test['recall_dircalc']
            roc_auc_test = value_pack_test['rocauc_pp_dircalc']
            
            acc_cv_test = value_pack_test['acc_crossval']
            prec_cv_test = value_pack_test['prec_crossval']
            recall_cv_test = value_pack_test['recall_crossval']
            roc_auc_cv_test = value_pack_test['rocauc_crossval']

            # ----------------
            # Permutation importance of features : probe which features are most predictive
            if fea == 0:
                feature_names = ['joy', 'joy1derv', 'joy2derv', 'fres', 'freq_t', 'freq_fres']
                ovtot = pipeline_permutation_importance(model, X_test, Y_test_1D, feature_names)
            else:
                ovtot.append('na')
            # ----------------

            # ----------------
            # Save all data to array 
            res_permod.append([ynum, m_name, fea, desired_col, acc_train, prec_train, recall_train, roc_auc_train, acc_test, prec_test, recall_test, roc_auc_test, acc_cv_test, prec_cv_test, recall_cv_test, roc_auc_cv_test, ovtot])
            # ----------------
            
            # Save data matrices to file per model result :
            file_name = "res_exp_%s_%s_%s_ynum%d_%s_fea%d.pkl" % (exp, ax_val, ss_val, ynum, m_name, fea)
            open_file = open(file_name, "wb")
            pickle.dump(res_permod, open_file)
            open_file.close()
            
    return

# Batch runs

In [ ]:
def run_main_script(exp, ax_val, ss_val):
    df = get_df(ax_val, ss_val)
    
    feat0, t_feat0, y1_feat0, y2_feat0, y3_feat0 = create_labels_and_initial_feature(df) 
    
    del df
    
    norm_feat0, norm_feat1, norm_feat2, norm_feat3, feat4, feat4_1 = feature_creation_preprocessing(feat0, t_feat0)
    
    del feat0, t_feat0
    
    y_alllabel = [y1_feat0, y2_feat0, y3_feat0]
    
    # Loop over the different y labels
    for ynum in range(len(y_alllabel)):
        print('ynum : ', ynum)
        
        # ----------------
        
        # Select y
        y_label = y_alllabel[ynum]
        
        # ----------------
        
        # Put the nested arrays into a DataFrame
        # Put the entire trial vector into a single DataFrame

        norm_feat0 = np.array(norm_feat0)
        # norm_feat0.shape is (442, 471), there are 442 trials
        # norm_feat0[0].shape is (471,) means that there are 471 data point in the first trial

        df_test = pd.DataFrame()
        for tr in range(norm_feat0.shape[0]):

            # way 1 : To put a vector into each DataFrame cell  
            # data = [norm_feat0[tr]], [norm_feat1[tr]], [norm_feat2[tr]], [norm_feat3[tr]], [feat4[tr]], [feat4_1[tr]], [y_label[tr]]
            
            data = [norm_feat0[tr]], [norm_feat1[tr]], [norm_feat2[tr]], [norm_feat3[tr]], [y_label[tr]]
            
            df_row = pd.DataFrame(data=data)
            df_row = df_row.T # or df1.transpose()

            df_test = pd.concat([df_test, df_row], axis=0)

        df_test = df_test.reset_index(drop=True)  # reset index : delete the old index column
        
        # Automatic label
        n_col = df_test.shape[1]
        dfl = {}
        for nf in range(n_col):
            if nf == n_col-1:
                dfl[nf] = 'y'
            else:
                dfl[nf] = '%d' % (nf)
        df_test = df_test.rename(dfl, axis=1)
        
        del data, df_row, y_label
        
        # ----------------
        
        df_test2 = pad_data_2makeclasses_equivalent(df_test)
        
        del df_test
        
        # ----------------
        
        # Explode the vectors per DataFrame
        featlab_new = explode_without_colnames2(df_test2)
        
        del df_test2
        
        # ----------------
        
        # classification for 3 partitions of feature data per 8 classifiers
        classify(featlab_new, ynum)
        
        del featlab_new

    return

In [ ]:
import warnings
warnings.filterwarnings('ignore')

exp = 'rot'
ax_val = 'all'
ss_val = 'all'
run_main_script(exp, ax_val, ss_val)

In [ ]:
# ------------------------------
# Manual runs
# Justification for not doing a loop : Can not do a loop because the computer stops or it stops 
# for weird reasons.  Have to run each manually.
# ------------------------------


# -------------
# DONE
# -------------

exp = 'rot'
ax_val = 'all'
ss_val = 'all'
run_main_script(exp, ax_val, ss_val)

exp = 'rot'
ax_val = 'all'
ss_val = 'sub'
run_main_script(exp, ax_val, ss_val)

exp = 'rot'
ax_val = 'all'
ss_val = 'sup'
run_main_script(exp, ax_val, ss_val)

# -------------

exp = 'trans'
ax_val = 'all'
ss_val = 'all'
run_main_script(exp, ax_val, ss_val)

exp = 'trans'
ax_val = 'all'
ss_val = 'sub'
run_main_script(exp, ax_val, ss_val)

exp = 'trans'
ax_val = 'all'
ss_val = 'sup'
run_main_script(exp, ax_val, ss_val)

# -------------

exp = 'rot'
ax_val = 'ax0'
ss_val = 'all'
run_main_script(exp, ax_val, ss_val)

exp = 'rot'
ax_val = 'ax1'
ss_val = 'all'
run_main_script(exp, ax_val, ss_val)

exp = 'rot'
ax_val = 'ax2'
ss_val = 'all'
run_main_script(exp, ax_val, ss_val)

# -------------

exp = 'trans'
ax_val = 'ax0'
ss_val = 'all'
run_main_script(exp, ax_val, ss_val)

exp = 'trans'
ax_val = 'ax1'
ss_val = 'all'
run_main_script(exp, ax_val, ss_val)

exp = 'trans'
ax_val = 'ax2'
ss_val = 'all'
run_main_script(exp, ax_val, ss_val)
